# You Only Look Once (YOLO)

Este componente utiliza o modelo YOLO para classificação usando [Yolov4](https://pypi.org/project/yolov4/). <br>
Está é uma biblioteca que encapsula o modelo yolov4 com algumas variações, podendo utilizar o modelo completo, como também,a versão reduzida.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

In [ ]:
# Download classes
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-yolo/coco.names

# Download weights
import gdown
gdown.download('https://drive.google.com/uc?id=1L-SO373Udc9tPz5yLkgti5IAXFboVhUt', 'yolov4.weights', quiet=False)

In [ ]:
%%writefile Model.py
from typing import List, Iterable, Dict, Union

import base64
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from yolov4.tf import YOLOv4


class Model:
    
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/yolo.joblib")
        self.names = artifacts["names"]
        
    
    def class_names(self):
        
        return ['class', 'probability', 'bboxcoord1', 'bboxcoord2', 'bboxcoord3', 'bboxcoord4']
    
    
    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:

        yolo = YOLOv4()
        yolo.classes = "coco.names"
        yolo.make_model()
        yolo.load_weights("yolov4.weights", weights_type="yolo")
        
        im_bytes = base64.b64decode(X[0,0])
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        frame = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        
        bboxes = yolo.predict(frame)
        
        result = []
        for box in bboxes:
            result.append(self.names[int(box[4])], box[5], box[0:4])
        
        return np.array(result)